In [1]:
from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder
import torch        

In [2]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device


## Autoencoder

In [3]:
### Initialization of the Autoencoder 
SEQ_LEN = 3000
HIDDEN_DIM = 512
ENCODING_SIZE = 64
model = AutoEncoder(vocab_size=100, embedding_size=HIDDEN_DIM, encoding_size=ENCODING_SIZE, sequence_len=SEQ_LEN)


/Users/carlomarx/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
#let's assume we have a batch of 2 people
x = torch.randint(1,99, size=(2,SEQ_LEN))
y = model(x) 
## returns the original shape


In [5]:
### only to use the encoder part 
y = model.encode(x) # here y contains embedding of a survey per row

## Squeeze - Excite Networks

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SqueezeExcitation(nn.Module):
    def __init__(self, num_columns, hidden_size, reduction_ratio=16):
        super(SqueezeExcitation, self).__init__()
        # Assuming the reduction happens across the hidden_size
        self.num_columns = num_columns
        self.reduced_size = max(1, hidden_size // reduction_ratio)
        
        self.squeeze = nn.AdaptiveAvgPool1d(1)  # Squeezes hidden_size to 1
        self.excitation = nn.Sequential(
            nn.Linear(num_columns, self.reduced_size),
            nn.ReLU(inplace=True),
            nn.Linear(self.reduced_size, num_columns),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: [batch_size, num_columns, hidden_size]
        # Squeeze operation
        z = self.squeeze(x.permute(0, 1,2))  # Change to [batch_size, hidden_size, num_columns] for pooling
        # z shape: [batch_size, hidden_size, 1]
        print(z.shape)
        # Excitation operation
        z = z.view(z.size(0), -1)  # Flatten [batch_size, hidden_size]
        print(z.shape)
        s = self.excitation(z)  # [batch_size, num_columns]
        print(s.shape)
        s = s.view(s.size(0), s.size(1), 1)  # Reshape to [batch_size, num_columns, 1] to match original dimensions
        
        return x * s  # Apply recalibration weights to the original input

# Example usage
se_block = SqueezeExcitation(SEQ_LEN, HIDDEN_DIM)

# Example input
output_tensor = se_block(model.embedding(x))
print(output_tensor.shape)  # Should be [5, 10, 512]


torch.Size([2, 3000, 1])
torch.Size([2, 3000])
torch.Size([2, 3000])
torch.Size([2, 3000, 512])


In [7]:
y

tensor([[1.0140, 1.0544, 0.5745, 1.1459, 1.0752, 1.0970, 1.2246, 1.0905, 1.1810,
         0.9075, 1.1066, 1.1111, 1.1480, 1.1564, 1.1017, 0.9926, 1.2691, 1.0983,
         1.1192, 1.1829, 1.0128, 1.0408, 1.0886, 1.1149, 1.1851, 1.1490, 1.2068,
         1.0217, 1.2480, 1.1243, 1.0000, 1.2659, 1.3011, 0.9569, 1.0429, 0.9876,
         1.0209, 1.1997, 1.1960, 1.1143, 1.1687, 1.1109, 1.0508, 1.0933, 1.0979,
         1.0760, 1.0780, 1.0757, 1.0974, 1.0552, 1.1843, 1.2101, 1.2528, 0.9350,
         1.0819, 1.0984, 1.1817, 1.1034, 0.9914, 1.0035, 1.0155, 1.2748, 1.1062,
         1.0720],
        [1.0825, 0.9838, 1.0101, 1.0756, 1.1222, 1.0585, 1.1174, 1.0644, 1.2160,
         0.9252, 1.1620, 1.0311, 1.0257, 0.9350, 1.0261, 1.0525, 1.1357, 0.9460,
         1.0744, 1.0142, 1.0255, 1.0425, 1.0415, 1.1484, 0.9303, 1.1661, 1.1607,
         1.0992, 1.0364, 1.2988, 1.0403, 1.0203, 1.1082, 1.1639, 1.1419, 0.6739,
         1.1018, 0.8879, 1.0156, 1.0770, 1.0094, 1.0933, 0.9685, 0.7231, 0.9469,
         0

## RNN

In [317]:
## 
# input_size -> the size of the embedding of the autoencoder model
# hidden_size -> the size of the RNN to use in the decoder (the input_size and hidden_size can be different)
model = GRUDecoder(input_size=6, hidden_size=10, max_seq_len=4).to(get_device())
loss_f = nn.BCEWithLogitsLoss()
solver = torch.optim.AdamW(model.parameters())

The model is going to set all input MASK to None
Using MPS (Metal Performance Shaders) device


In [318]:
# This is just an example

MAX_SEQ_LEN = 4 # max number of surveyas a person (in our dataset can have)
INPUT_SIZE = 6 # hidden dimmensions of autoencodder.

# let's say we have a person who only have 3 surveys
x0 = torch.rand(INPUT_SIZE) # embedding for the 1st survey 
x1 = torch.rand(INPUT_SIZE) # embedding for the 2nd survey
x2 = torch.rand(INPUT_SIZE)
x3 = torch.rand(INPUT_SIZE)
# the tensor for the person should be on the shape [MAX_SEQ_LEN, INPUT_SIZE]

e = torch.zeros(MAX_SEQ_LEN, INPUT_SIZE)
e[0] = x0
e[1] = x1
e[2] = x2
e = e.to(get_device()) # so this is a tensor for the person
#we also need to specify that the sequence has 'empty' embeddings
#mask = torch.BoolTensor([True, True, False, False]).to(get_device()) # the last two dimensions are empty
## it is important that you append existing survey embeddings right next to each other (even if the year is missign between them, they should be still appended one after another)

## let assume we have a batch of people, I am reusing the same person, but in the pipeline is should be different people
# the batch size is 3 here 

x = torch.stack([e,e,e])
y = torch.tensor([1.,1,1.]).to(get_device())
#mask = torch.stack([mask, mask, mask])

Using MPS (Metal Performance Shaders) device
Using MPS (Metal Performance Shaders) device
Using MPS (Metal Performance Shaders) device


In [319]:
for i in range(100):
    solver.zero_grad()
    #xx = torch.nn.functional.sigmoid(model(x, mask))
    xx = model(x, None)

    loss = loss_f(xx.view(-1), y.view(-1))
    loss.backward()
    solver.step()

In [320]:
loss

tensor(0.2001, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)